In [1]:
from html5lib.constants import adjustMathMLAttributes
%pip install bs4  matplotlib
#%pip install --upgrade pip
%pip install yfinance
%pip install html5lib
%pip install pandas openpyxl



[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import html5lib

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
def plot_graph(stock_data , reveneue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Stock Price ($)", "Historical Revenue ($)"), vertical_spacing=0.5)
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data.Date, infer_datetime_format=True), y=stock_data.Close.astype("float"), name = "Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(reveneue_data.Date, infer_datetime_format = True), y = reveneue_data.Revenue.astype("float"), name = "Revenue"), row= 2, col=1)
    # Since the x-axes are shared, we only need to update the title on the bottom chart.
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text = "Price ($)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($ Millions)", row = 2, col=1)
    fig.update_layout(showlegend=False, height = 1000, title= stock, xaxis_rangeslider_visible=True)
    fig.show()

In [4]:
#yfiance for extracting stock info

In [5]:
#Using the Ticker function to create a ticker object
tesla_data = yf.Ticker("TSLA")
#history function helps to extract stock info
#setting period parameter to max to get information for the maximum amount of time
tsla_data = tesla_data.history(period="max")
tsla_data.reset_index(inplace=True)
tsla_data.head(100)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0
...,...,...,...,...,...,...,...,...
95,2010-11-11 00:00:00-05:00,1.906667,1.940000,1.822000,1.869333,29179500,0.0,0.0
96,2010-11-12 00:00:00-05:00,1.883333,2.033333,1.871333,1.989333,40936500,0.0,0.0
97,2010-11-15 00:00:00-05:00,2.014667,2.196000,2.014667,2.053333,39343500,0.0,0.0
98,2010-11-16 00:00:00-05:00,2.066667,2.093333,1.894667,1.978000,20214000,0.0,0.0


In [6]:
#Webscraping to extract TESLA revenue data

In [7]:
#using requests library to download the webpage
url="https://macrotrends.net/stocks/charts/TSLA/tesla/revenue"
# To prevent being blocked by the website, we'll add a User-Agent header to our request
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
#save the text for the response
response = requests.get(url, headers=headers)
html_text = ""
if response.status_code == 200:
    html_text = response.text
else:
    print(f"Failed to retrieve page, status code: {response.status_code}")

soup = BeautifulSoup(html_text, "html5lib")


In [8]:
#Using bs extract the table with TESLA Quarterly Revenue
revenue_table = None
for table in soup.find_all('table'):
    if 'Tesla Quarterly Revenue' in str(table):
        revenue_table = table
        break

if revenue_table:
    # Using a more modern way to build the DataFrame
    rows_list = []
    for row in revenue_table.tbody.find_all("tr"):
        col = row.find_all("td")
        if col: 
            date = col[0].text 
            #to remove comma and dollar sign 
            revenue = col[1].text.replace("$", "").replace(",", "")
            rows_list.append({"Date": date, "Revenue": revenue})
    tsla_revenue = pd.DataFrame(rows_list)
else:
    print("Revenue table not found.")
    tsla_revenue = pd.DataFrame(columns=["Date", "Revenue"]) # Create an empty dataframe

#show the df
tsla_revenue

,Date,Revenue
0,2025-12-31,24901
1,2025-09-30,28095
2,2025-06-30,22496
3,2025-03-31,19335
4,2024-12-31,25707
5,2024-09-30,25182
6,2024-06-30,25500
7,2024-03-31,21301
8,2023-12-31,25167
9,2023-09-30,23350


In [9]:
#removing null values
tsla_revenue = tsla_revenue[tsla_revenue["Revenue"]!=""]
tsla_revenue

,Date,Revenue
0,2025-12-31,24901
1,2025-09-30,28095
2,2025-06-30,22496
3,2025-03-31,19335
4,2024-12-31,25707
5,2024-09-30,25182
6,2024-06-30,25500
7,2024-03-31,21301
8,2023-12-31,25167
9,2023-09-30,23350


In [10]:
plot_graph(tsla_data, tsla_revenue, "TESLA Historical Stock Price ($) and Revenue")

/var/folders/w3/dr79nrrd49s6kd7trrw92kbh0000gn/T/ipykernel_42284/2737859960.py:3: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

/var/folders/w3/dr79nrrd49s6kd7trrw92kbh0000gn/T/ipykernel_42284/2737859960.py:4: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



In [11]:
#MA Stock Data


In [12]:
Mastercard=yf.Ticker("MA")
MA_data = Mastercard.history(period="max")
MA_data.reset_index(inplace=True)
MA_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2006-05-25 00:00:00-04:00,3.656686,4.178421,3.647612,4.173884,395343000,0.0,0.0
1,2006-05-26 00:00:00-04:00,4.201108,4.241032,4.002395,4.076799,103044000,0.0,0.0
2,2006-05-30 00:00:00-04:00,4.080426,4.081334,3.888065,3.992412,49898000,0.0,0.0
3,2006-05-31 00:00:00-04:00,4.024170,4.115814,4.024170,4.077704,30002000,0.0,0.0
4,2006-06-01 00:00:00-04:00,4.076798,4.364433,4.074076,4.310899,62344000,0.0,0.0


In [13]:
#using requests library to download the webpage
url="https://www.macrotrends.net/stocks/charts/MA/mastercard/revenue"
# To prevent being blocked by the website, we'll add a User-Agent header to our request
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
#save the text for the response
response = requests.get(url, headers=headers)
html_text = ""
if response.status_code == 200:
    html_text = response.text
else:
    print(f"Failed to retrieve page, status code: {response.status_code}")

soup = BeautifulSoup(html_text, "html5lib")


In [14]:
#Using bs the Quarterly Revenue
revenue_table = None
for table in soup.find_all('table'):
    if 'Mastercard Quarterly Revenue' in str(table):
        revenue_table = table
        break

if revenue_table:
    # Using the modern way to build the DataFrame to avoid the 'append' error.
    rows_list = []
    for row in revenue_table.tbody.find_all("tr"):
        col = row.find_all("td")
        if col:
            date = col[0].text
            #to remove comma and dollar sign
            revenue = col[1].text.replace("$", "").replace(",", "")
            rows_list.append({"Date": date, "Revenue": revenue})
    ma_revenue = pd.DataFrame(rows_list)
else:
    print("Mastercard revenue table not found.")
    ma_revenue = pd.DataFrame(columns=["Date", "Revenue"]) # Create an empty dataframe

#show the df
ma_revenue.head()


,Date,Revenue
0,2025-12-31,8806
1,2025-09-30,8602
2,2025-06-30,8133
3,2025-03-31,7250
4,2024-12-31,7489


In [15]:
ma_revenue = ma_revenue[ma_revenue["Revenue"]!=""]
ma_revenue

,Date,Revenue
0,2025-12-31,8806
1,2025-09-30,8602
2,2025-06-30,8133
3,2025-03-31,7250
4,2024-12-31,7489
5,2024-09-30,7369
6,2024-06-30,6961
7,2024-03-31,6348
8,2023-12-31,6548
9,2023-09-30,6533


In [16]:
plot_graph(MA_data, ma_revenue, "Mastercard Historical Stock Price ($) and Revenue")


/var/folders/w3/dr79nrrd49s6kd7trrw92kbh0000gn/T/ipykernel_42284/2737859960.py:3: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

/var/folders/w3/dr79nrrd49s6kd7trrw92kbh0000gn/T/ipykernel_42284/2737859960.py:4: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

